In [17]:
import json
import shapely.geometry

Read in several-Mb-large sidewalks geojson

In [18]:
with open('../data/sidewalks_data.geojson') as f:
    cd_fc = json.load(f)

Get just the sidewalks in the West Woodland neighborhood

In [19]:
# West, South, East, North coordinates
#greenlake_wsen = [-122.334073,47.674548,-122.319094,47.68183]
cd_box = shapely.geometry.Polygon([[-122.3046970367,47.5979667018],
                                   [-122.3046970367,47.6006000885],
                                   [-122.2997403145,47.6006000885],
                                   [-122.2997403145,47.5979667018],
                                   [-122.3046970367,47.5979667018]])

cd_fc = {'type': 'FeatureCollection', 'features': []}
for feature in sidewalk_fc['features']:
    if cd_box.intersects(shapely.geometry.shape(feature['geometry'])):
        feature['properties']['fill-color'] = '#eee'
        feature['properties']['stroke-color'] = '#000'
        feature['properties']['stroke-opacity'] = 1
        feature['properties']['fill-opacity'] = 1
        cd_fc['features'].append(feature)

Write to file

In [20]:
with open('../data/sidewalks_data_cd.geojson', 'w') as g:
    json.dump(cd_fc, g)

Combine with places where heuristic model disagrees with crowdsourcing

In [21]:
with open('../cse546/learndata-errorsHeuristic_onCrowd.geojson') as h:
    error_dat = json.load(h)

for feature in error_dat['features']:
    if cd_box.intersects(shapely.geometry.shape(feature['geometry'])):
        feature['properties']['opacity'] = 1
        cd_fc['features'].append(feature)

In [22]:
with open('../figures/figure-heuristic-errors.geojson', 'w') as i:
    json.dump(cd_fc, i)